## SnowPit Data - XML data to python dataframe
#### Alexis Emerson

In [1]:
import xmltodict
import pandas as pd

In [2]:
file_path = "../raw_data/sp/all-MT.xml"
with open(file_path, "rb") as file:
    pit_data = xmltodict.parse(file)

In [3]:
# pit_data

In [4]:
df = pd.DataFrame.from_dict(pit_data['Pit_Data']['Pit_Observation']).set_index('@nid')

In [5]:
df

,@activities,@iLayerNumber,@iDepth,@bld,@stability,@aviPit,@crownObs,@aviLoc,@incline,@winDir,...,Location,Layer,Shear_Test_Result,Density_Profile,Temperature_Profile,@bootPen,@skiPen,@surfGrainType,@surfGrainSize,@tempUnits
@nid,,,,,,,,,,,,,,,,,,,,,
45338,,4,65,7.91,Good,false,false,,17,E,...,"{'@ID': '', '@type': 'LATLONG', '@zone': '12T'...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'CT', '@sdepth': '65.0', '@score': ...","{'@profile': '', '@densityUnits': 'kg_cubic_m'...","{'@temp_profile': '', '@depthUnits': 'cm', '@t...",NaN,NaN,NaN,NaN,NaN
45310,Ski tracks on slope; We skied slope;,2,64,7.91,Good,false,false,,,SW,...,"{'@ID': '', '@type': 'LATLONG', '@zone': '12T'...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '20.0', '@score':...","{'@profile': '', '@densityUnits': 'kg_cubic_m'...","{'@temp_profile': '', '@depthUnits': 'cm', '@t...",NaN,NaN,NaN,NaN,NaN
45286,Recent avalanche activity on similar slopes; R...,6,50,7.91,Poor,false,false,,25,SW,...,"{'@ID': '', '@type': 'LATLONG', '@zone': '', '...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '64.0', '@score':...","{'@profile': '', '@densityUnits': 'kg_cubic_m'...","{'@temp_profile': '', '@depthUnits': 'cm', '@t...",NaN,NaN,NaN,NaN,NaN
45347,,6,22,7.91,,false,false,,10,S,...,"{'@ID': '', '@type': 'LATLONG', '@zone': '', '...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '98.0', '@score':...","{'@profile': '', '@densityUnits': 'kg_cubic_m'...","{'@temp_profile': '', '@depthUnits': 'cm', '@t...",NaN,NaN,NaN,NaN,NaN
45346,,6,22,7.91,,false,false,,10,S,...,"{'@ID': '', '@type': 'LATLONG', '@zone': '', '...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '98.0', '@score':...","{'@profile': '', '@densityUnits': 'kg_cubic_m'...","{'@temp_profile': '', '@depthUnits': 'cm', '@t...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32437,,,,7.77,,false,false,,5,,...,"{'@ID': '', '@type': 'LATLONG', '@zone': '12T'...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '180.0', '@depthU...",{'@profile': '195.0_156?185.0_252?175.0_240?16...,NaN,30,NaN,NaN,NaN,C
32430,,10,45,7.77,Good,false,false,,30,,...,"{'@ID': '', '@type': 'LATLONG', '@zone': '12T'...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","{'@code': 'ECT', '@sdepth': '45.0', '@depthUni...","{'@profile': '', '@depthUnits': 'cm', '@densit...",NaN,NaN,NaN,NaN,NaN,C
32427,We skied slope;,8,0,7.77,Good,false,false,,27,,...,"{'@ID': '', '@type': 'LATLONG', '@zone': '12T'...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","{'@code': 'ECT', '@sdepth': '', '@depthUnits':...","{'@profile': '', '@depthUnits': 'cm', '@densit...",NaN,NaN,NaN,NaN,NaN,F


In [11]:
layers = df.Layer.to_frame()
layers

,Layer
@nid,
45338,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'..."
45310,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'..."
45286,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'..."
45347,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'..."
45346,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'..."
...,...
32437,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'..."
32430,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'..."
32427,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'..."


In [12]:
for l in layers:
    for d in l:
        ph = pd.DataFrame.from_dict(data=d, orient='index')
        ph.index.name = 't'
        ph.columns = ['val']
#         df.pivot(ph, columns='t', values=['0'])
        print(ph)
#         dflayers = pd.concat([dflayers, ph], axis=1)

AttributeError: 'str' object has no attribute 'values'